In [204]:
from lib import db_config
import plotly_express as px

import pandas as pd
import string
import re

# Mapping cohorts without looking at the PANs, but year and papercodes

In [135]:
# reading Dipak's data dump
dtype = {
 'school_code' : int,
 'school_name' : str,
 'city' : str,
 'pincode' : str,
 'state' : str,
 'country' : str,
 'season' : str,
 'year' : str,
 'class': np.uint8,
 'studentCount': int,
 'subject' : str,
 'papercode': str,
 'skill_name' : str,
 'sub_skill_name' : str,
 'subSkillLevelAccuracy': float,
 'subSkillLevelQuestionCount' : np.uint32,
 'skillLevelAccuracy' : float,
 'skillLevelQuestionCount' : np.uint32,
 'totalAccuracy' : float,
 'totalQuestionCount': np.uint32,
 'national_studentCount': int,
 'national_subSkillLevelAccuracy' : float,
 'totalStandardDeviation' : float,
 'national_skillLevelAccuracy' : float,
 'national_studentCount' : int,
 'national_totalAccuracy' : float,
 'national_totalStandardDeviation' : float,
 'national_skillLevelStandardDeviation': float}
file_path = '../../../Desktop/EI/From others/Dipak/Parent Connect/data2'
df = pd.read_csv(file_path+'/final_asset_school_accuracy.csv', dtype = dtype)

df['total_delta'] = df['totalAccuracy'] - df['national_totalAccuracy']
df['skill_delta'] = df['skillLevelAccuracy'] - df['national_skillLevelAccuracy']
df['subSkill_delta'] = df['subSkillLevelAccuracy'] - df['national_subSkillLevelAccuracy']

#Mapping to boards
df_school_board = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR_AO8CCe9wrCqYyXAKutE4NceqSuTZuWQdaZoxhLwPOBQYAXOIEZzXL1xsYMWX3uzhi9rtbNWk67PW/pub?gid=288331316&single=true&output=csv')
df = pd.merge(df, df_school_board, on='school_code', how = 'left')

In [311]:
#Choosing the school to analyse

school_code = 173428
# school_code = 46067
min_cohort_length = 1
df_school = df[df['school_code'] == school_code]

school_name = df_school.school_name.value_counts().index[0]
school_board = df_school.primary_board.value_counts().index[0]
school_city = df_school.city.value_counts().index[0]

In [312]:
## setup for t
# rounds = db_config.find_rounds(df_school)
# subjects = db_config.find_subjects(df_school)
# classes = db_config.find_classes(df_school)
# class_rounds = db_config.find_class_rounds(df_school)

# class_rounds.sort(reverse=True)
# class_rounds

# # ordered list of rounds
# r = list(string.digits)
# r.remove('0') 
# r += list(string.ascii_uppercase) + ['#']

# #ordered list of classes
# c = list(string.digits) + ['A', "B"]
# c.remove('0')

This funciton finds the number of cohorts that have likelyhood of having the same students

In [313]:
## This has been converted into a function
# # ordered list of rounds
# r = list(string.digits)
# r.remove('0') 
# r += list(string.ascii_uppercase) + ['#']

# #ordered list of classes
# c = list(string.digits) + ['A', "B"]
# c.remove('0')

# class_rounds = db_config.find_class_rounds(df_school)
# class_rounds.sort(reverse=True)

# temp = class_rounds
# cohorts = []
# cohort_long = []
# for i in temp:
#     cls, rnd = i
#     temp_cls = c[c.index(cls):]
#     temp_rnd = r[r.index(rnd)::2]
#     co = list(zip(temp_cls, temp_rnd))
#     co = [''.join(x) for x in co]

#     cohort = [x for x in co if x in class_rounds]
# #     print( co, cohort)
#     cohorts.append(cohort)
# #     print(cohorts,"--" )
#     temp = list(set(temp) - set(cohort))
# #     break

# cohort_long = [x for x in cohorts if len(x) > 1]

# cohort_long

In [314]:
# takes a dataframe of a a school's data and finds trains of cohorts

def find_cohorts(df_school):
    # ordered list of rounds
    r = list(string.digits)
    r.remove('0') 
    r += list(string.ascii_uppercase) + ['#']

    #ordered list of classes
    c = list(string.digits) + ['A', "B"]
    c.remove('0')
    
    #combinations of tests in class rounds in that school that has happened
    class_rounds = db_config.find_class_rounds(df_school)
    class_rounds.sort(reverse=True)

    temp = class_rounds
    cohorts = []
    cohort_long = []
    for i in temp:
        cls, rnd = i
        temp_cls = c[c.index(cls):]
        temp_rnd = r[r.index(rnd)::2]
        co = list(zip(temp_cls, temp_rnd))
        co = [''.join(x) for x in co]

        cohort = [x for x in co if x in class_rounds]
    #     print( co, cohort)
        cohorts.append(cohort)
    #     print(cohorts,"--" )
        temp = list(set(temp) - set(cohort))

    return [x for x in cohorts if len(x) > min_cohort_length]

In [315]:
cohorts = find_cohorts(df_school)

In [316]:
cohorts

[['8S', 'AW'],
 ['8D', '9F'],
 ['7W', '9#'],
 ['7S', '9W'],
 ['7D', '8F'],
 ['6W', '8#'],
 ['6S', '8W'],
 ['6D', '7F'],
 ['5W', '7#'],
 ['5S', '7W', '9#'],
 ['5I', 'AS'],
 ['5D', '6F'],
 ['4W', '6#'],
 ['4S', '6W', '8#'],
 ['4I', '9S'],
 ['4D', '5F'],
 ['3W', '5#'],
 ['3S', '5W', '7#'],
 ['3I', '8S', 'AW'],
 ['3D', '4F']]

In [292]:
# # one way of selecting the table
# df_school.papercode[df_school.papercode.str.contains('|'.join(cohorts[9]))].value_counts()

In [317]:
## testing the function to be able to genreate plot of the progress

# i = ['8S', 'AW']
i = cohorts[0]
# forms a regex to choose the appropriate papercodes
j = lambda x :'.'+'('+'|'.join(x)+')'
# returns true or false based on whether the paper code contains the cohort
present = lambda x : bool(re.search(j(i), x))

temp = df_school[df_school.papercode.apply(present)].drop_duplicates(['papercode', 'school_code'])
temp.year = temp.year.astype(str)
px.scatter(
    temp,
    x = 'year',
    y = 'total_delta',
    color = 'subject',
    size = 'studentCount',
    text = ['class', 'subject'], 
    trendline = 'lowess',
    range_x = [2007, 2019],
    title = "Year on Year performance of same students as they progress across the years"
)
#     break

Looping over the data to label the cohorts in each school

In [318]:
data, temp = pd.DataFrame(),pd.DataFrame()
for ix, i in enumerate(cohorts):
    # forms a regex to choose the appropriate papercodes
    j = lambda x :'.'+'('+'|'.join(x)+')'
    # returns true or false based on whether the paper code contains the cohort
    present = lambda x : bool(re.search(j(i), x))
    temp = df_school[df_school.papercode.apply(present)]
    temp.year = temp.year.astype(str)
    temp['cohort_number'] = ix +1
    temp.cohort_number = temp.cohort_number.astype(int)
    data  = data.append(temp)

/Users/manghat/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/manghat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [319]:
for i, j in enumerate(cohorts):
    print("Cohort Number :", i+1, "Rounds :", j)
px.scatter(
    data.sort_values('year'),#.drop_duplicates(['papercode']),
    x = 'year', 
#     range_x = [2001,2019],
#     range_y = [-10, 30],
    y = 'total_delta',
    color = 'subject',
    facet_row = 'cohort_number',
    size = 'studentCount',
    text = 'class', 
    trendline = 'lowless',
    title = "Year on Year performance of same students as they progress across the years",
    height = 5000,
    size_max = 10
)

Cohort Number : 1 Rounds : ['8S', 'AW']
Cohort Number : 2 Rounds : ['8D', '9F']
Cohort Number : 3 Rounds : ['7W', '9#']
Cohort Number : 4 Rounds : ['7S', '9W']
Cohort Number : 5 Rounds : ['7D', '8F']
Cohort Number : 6 Rounds : ['6W', '8#']
Cohort Number : 7 Rounds : ['6S', '8W']
Cohort Number : 8 Rounds : ['6D', '7F']
Cohort Number : 9 Rounds : ['5W', '7#']
Cohort Number : 10 Rounds : ['5S', '7W', '9#']
Cohort Number : 11 Rounds : ['5I', 'AS']
Cohort Number : 12 Rounds : ['5D', '6F']
Cohort Number : 13 Rounds : ['4W', '6#']
Cohort Number : 14 Rounds : ['4S', '6W', '8#']
Cohort Number : 15 Rounds : ['4I', '9S']
Cohort Number : 16 Rounds : ['4D', '5F']
Cohort Number : 17 Rounds : ['3W', '5#']
Cohort Number : 18 Rounds : ['3S', '5W', '7#']
Cohort Number : 19 Rounds : ['3I', '8S', 'AW']
Cohort Number : 20 Rounds : ['3D', '4F']


# TODO 
- loop around the cohorts to plot it all in the same graph
